In [ ]:
# --- 0. Instalación y Setup ---
!pip install roboflow ultralytics -q
from ultralytics import YOLO
import os
import yaml
from roboflow import Roboflow
from google.colab import drive
from IPython.display import clear_output, display
import pandas as pd

# --- 1. CONFIGURACIÓN PRINCIPAL ---
# Escribe el nombre de la carpeta del entrenamiento que quieres evaluar:
TRAINING_RUN_NAME = 'Coffee_Seg_L-Model_StableAug_1088px_Batch16_v1'

# Configuración de tu API de Roboflow
ROBOFLOW_API_KEY = "tZR7XZhFvySrbVM3hfH4"
ROBOFLOW_WORKSPACE = "luchini"
ROBOFLOW_PROJECT = "datasetenproceso-irzcf"
# Versión del dataset que usarás para VALIDAR (tu "set de prueba" estándar)
VALIDATION_DATASET_VERSION = 15

print("Configuración cargada. Montando Google Drive...")
drive.mount('/content/drive')
print("✅ Drive Montado.")

# --- 2. Construcción de Rutas y Verificación ---
base_drive_path = '/content/drive/MyDrive/YOLOv8_Training'
run_path = os.path.join(base_drive_path, TRAINING_RUN_NAME)
model_path = os.path.join(run_path, 'weights/best.pt')
args_path = os.path.join(run_path, 'args.yaml')

print(f"\nBuscando archivos para el run: {TRAINING_RUN_NAME}...")

all_files_found = True
if not os.path.exists(run_path):
    print(f"❌ Error: La carpeta del run no existe en: {run_path}")
    all_files_found = False
if not os.path.exists(model_path):
    print(f"❌ Error: No se encontró 'best.pt' en: {model_path}")
    all_files_found = False
if not os.path.exists(args_path):
    print(f"❌ Error: No se encontró 'args.yaml' en: {args_path}")
    all_files_found = False

if not all_files_found:
    print("\nDeteniendo ejecución. Revisa el nombre de la carpeta y los archivos.")
else:
    print("✅ Archivos 'best.pt' y 'args.yaml' encontrados.")

    # --- 3. Carga de Hiperparámetros (en memoria) ---
    # (Los cargamos ahora, pero los mostraremos después de limpiar la pantalla)
    with open(args_path, 'r') as f:
        args = yaml.safe_load(f)

    # En lugar de una lista manual, convertimos TODO el diccionario 'args'
    # en un DataFrame de Pandas para mostrarlo completo.

    # 1. Convierte el diccionario de argumentos en una "Serie" de Pandas
    args_series = pd.Series(args, name="Valor")
    # 2. Convierte la Serie en un DataFrame
    hparams_df = args_series.reset_index()
    # 3. Renombra las columnas para que queden limpias
    hparams_df.columns = ["Parámetro", "Valor"]

    # (El 'hparams_data = {...}' manual de antes se elimina)

    # --- 4. Descarga del Dataset de VALIDACIÓN ---
    print(f"\nDescargando dataset de validación (v{VALIDATION_DATASET_VERSION}) desde Roboflow...")
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
    version = project.version(VALIDATION_DATASET_VERSION)
    dataset = version.download("yolov8")

    validation_data_yaml_path = f'{dataset.location}/data.yaml'

    with open(validation_data_yaml_path, 'r') as f:
        data_yaml = yaml.safe_load(f)
    data_yaml['train'] = f"{dataset.location}/train/images"
    data_yaml['val'] = f"{dataset.location}/valid/images"
    if os.path.exists(f"{dataset.location}/test/images"):
        data_yaml['test'] = f"{dataset.location}/test/images"
    with open(validation_data_yaml_path, 'w') as f:
        yaml.dump(data_yaml, f)

    # --- 5. LIMPIEZA Y RESUMEN ---
    clear_output(wait=True)
    print("✅ Drive Montado.")
    print(f"✅ Dataset de Validación (v{VALIDATION_DATASET_VERSION}) descargado y listo.")
    print(f"\nIniciando evaluación para el run: {TRAINING_RUN_NAME}")

    # --- Mostramos la tabla COMPLETA de Hiperparámetros ---
    print("\n--- Hiperparámetros Completos del Entrenamiento ---")
    display(hparams_df)

    # --- 6. Carga del Modelo y Ejecución de Validación ---
    print(f"\nCargando el modelo: {model_path}...")
    model = YOLO(model_path)
    print("✅ Modelo cargado.")

    print(f"\nEjecutando validación contra el dataset v{VALIDATION_DATASET_VERSION} (split='val')...")
    metrics = model.val(
        data=validation_data_yaml_path,
        split='val',
        imgsz=args.get('imgsz', 1088),
        batch=args.get('batch', 16)
    )
    print("\n✅ Validación completada.")

    # --- 7. Imprimir Métricas Clave ---
    print("\n--- Métricas de Rendimiento (Segmentation) ---")

    metrics_data = {
        "Métrica": [
            "Mask mAP50-95 (principal)", "Mask mAP50", "Mask mAP75",
            "Box mAP50-95", "Box mAP50", "Box mAP75"
        ],
        "Valor": [
            f"{metrics.seg.map:.4f}", f"{metrics.seg.map50:.4f}", f"{metrics.seg.map75:.4f}",
            f"{metrics.box.map:.4f}", f"{metrics.box.map50:.4f}", f"{metrics.box.map75:.4f}"
        ]
    }
    metrics_df = pd.DataFrame(metrics_data)
    display(metrics_df)

    print(f"\nResultados guardados en la carpeta del modelo: {run_path}")

✅ Drive Montado.
✅ Dataset de Validación (v15) descargado y listo.

Iniciando evaluación para el run: Coffee_Seg_L-Model_StableAug_1088px_Batch16_v1

--- Hiperparámetros Completos del Entrenamiento ---


,Parámetro,Valor
0,task,segment
1,mode,train
2,model,yolov8l-seg.pt
3,data,/content/DatasetEnProceso-15/data.yaml
4,epochs,500
...,...,...
101,auto_augment,randaugment
102,erasing,0.4
103,cfg,None
104,tracker,botsort.yaml



Cargando el modelo: /content/drive/MyDrive/YOLOv8_Training/Coffee_Seg_L-Model_StableAug_1088px_Batch16_v1/weights/best.pt...
✅ Modelo cargado.

Ejecutando validación contra el dataset v15 (split='val')...
Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
YOLOv8l-seg summary (fused): 125 layers, 45,915,743 parameters, 0 gradients, 210.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1097.1±566.3 MB/s, size: 47.2 KB)
val: Scanning /content/DatasetEnProceso-15/valid/labels... 67 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 67/67 503.7it/s 0.1s
val: /content/DatasetEnProceso-15/valid/images/_DSC0513_JPG.rf.c744f2f1b61040726f3a34f62746deef.jpg: 1 duplicate labels removed
val: New cache created: /content/DatasetEnProceso-15/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 9/9 2.5it/s 3.6s
                   a

,Métrica,Valor
0,Mask mAP50-95 (principal),0.8785
1,Mask mAP50,0.9176
2,Mask mAP75,0.9123
3,Box mAP50-95,0.8803
4,Box mAP50,0.9176
5,Box mAP75,0.9081



Resultados guardados en la carpeta del modelo: /content/drive/MyDrive/YOLOv8_Training/Coffee_Seg_L-Model_StableAug_1088px_Batch16_v1
No charts were generated by quickchart
